In [23]:
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Load model
https://huggingface.co/luqh/ClinicalT5-large

In [28]:
MODEL_ID = "luqh/ClinicalT5-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, model_max_length = 1024)
model = T5ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    from_flax = True, 
    low_cpu_mem_usage = False,
)

All Flax model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the Flax model and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

# Some medical text

Not that this will fail if there are too many tokens. We'll have to do some chunking or RAG in order to
deal with the small context window.

In [25]:
medical_text = """
Non-steroidal anti-inflammatory drugs are not only potent analgesics and antipyretics but also nephrotoxins, and may cause 
electrolyte disarray. In addition to the commonly expected effects, including hyperkalemia, hyponatremia, acute renal injury, 
renal cortical necrosis, and volume retention, glomerular disease with or without nephrotic syndrome or nephritis can occur as 
well including after years of seemingly safe administration. Minimal change disease, secondary membranous glomerulonephritis, 
and acute interstitial nephritis are all reported glomerular lesions seen with non-steroidal anti-inflammatory use. We report a 
patient who used non-steroidal anti-inflammatory drugs for years without diabetes, chronic kidney disease, or proteinuria; he 
then developed severe nephrotic range proteinuria with 7 g of daily urinary protein excretion. Renal biopsy showed minimal 
change nephropathy, a likely secondary membranous glomerulonephritis, and acute interstitial nephritis present simultaneously
in one biopsy. 

"""

In [26]:
len(medical_text)

1030

# Inference

In [27]:
prompt = f"summarize: {medical_text.strip()}"
inputs = tokenizer(prompt, return_tensors = "pt").to(model.device)

with torch.no_grad():
    summary_ids = model.generate(**inputs,
        max_new_tokens = 128, 
        num_beams = 4, 
        length_penalty = 1.5, 
        early_stopping = False, 
        no_repeat_ngram_size = 3, 
        encoder_no_repeat_ngram_size = 3)

RuntimeError: Tensor.item() cannot be called on meta tensors

In [ ]:
for summary_id in summary_ids:
    summary = tokenizer.decode(summary_id, skip_special_tokens = True)
    print(summary)